Great Lakes notebook for testing HVG code

First generate cross validation splits, then calculate HVGs and scores

In [ ]:
# Installing packages
# !pip install scanpy[skmisc]
# !pip install xgboost
# !pip install shap

In [1]:
import sys
sys.path.insert(1, '/home/gylam/siads699/Cancer_Prediction_10x')

In [2]:
from main_functions import *

import pandas as pd
import numpy as np
import pickle
import json

import scanpy as sc
from xgboost import XGBClassifier
import shap

import matplotlib.pyplot as plt
import seaborn as sns

/sw/pkgs/arc/mamba/py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Constants
RANDOM_STATE = 42
CV = 5

In [ ]:
# Load training dataset into anndata object
#adata = create_adata_train('EMTAB8107_2102-Breastcancer_counts','BRCA_EMTAB8107_expression.h5','EMTAB8107_2103-Breastcancer_metadata.csv.gz')

# Save adata to h5ad
#adata.write_h5ad('adata.h5ad')

In [4]:
adata = sc.read_h5ad('adata.h5ad')
print(adata)

AnnData object with n_obs × n_vars = 33043 × 22830
    obs: 'nGene', 'nUMI', 'CellFromTumor', 'PatientNumber', 'TumorType', 'TumorSite', 'CellType', 'orig_cancer_label'
    var: 'gene_ids', 'feature_types', 'genome'
    layers: 'norm', 'raw'


In [5]:
# Variables for training
#clf = XGBClassifier(max_depth = 4, n_estimators = 50, random_state = RANDOM_STATE)
clf = XGBClassifier(eta = 0.3, max_depth = 2, n_estimators = 100, random_state = RANDOM_STATE) #, max_iter = 500
num_features = [20, 50, 100, 200, 300, 400, 500, 1000, 2000, 5000]
#num_features = [20, 50, 100, 200, 300]
feat_sel_list = ['dge', 'seurat_v3', 'pearson_residuals', 'seurat', 'cell_ranger', 'random_all_genes'] # 'random_per_num'

In [ ]:
# For XGBoost, compare different numbers of features and different methods for selecting highly variable genes

results_df, fold_index, feat_order, shap_results = train_feat_loop_cv(clf, adata, 'PatientNumber', num_features, feat_sel_list,
                       random_state = RANDOM_STATE, k_fold = CV)
print(results_df.shape)
display(results_df.head())
results_df.to_csv('results_df_20241118_withdge.csv')

In [ ]:
# Plot performance metrics
results_df_pivot, fig= make_line_plots_metrics(results_df)
results_df_pivot.to_csv('results_df_pivot_20241118.csv')
fig.savefig('metrics_comparison_20241118.png')

In [ ]:
# Revise metrics plot - 4 metrics, larger fonts

results_df_sub = results_df[['feat_sel_type', 'num_features', 'fold', 'f1', 'accuracy', 'recall', 'precision']]
results_df_tall = results_df_sub.melt(id_vars=['feat_sel_type', 'num_features', 'fold'], var_name='metric', value_name='score')
display(results_df_tall.head())

# Save dataframe summarizing mean and stdev
results_df_pivot = pd.pivot_table(results_df_tall,
                                values=['score'],
                                index = ['feat_sel_type', 'num_features'],
                                columns = ['metric'],
                                aggfunc=['mean', 'std'])

# Plot 1 figure with all test metrics versus number of features - Facet by metric. Color by feature type

#sns.set_theme(style='white')
#with sns.plotting_context(font_scale=1.5):
g = sns.catplot(
  data=results_df_tall,
  x='num_features', y='score', col='metric',
  hue = 'feat_sel_type', col_wrap = 4, kind='point', capsize = 0.2,
  sharex = False, alpha = 0.7
)
g.savefig('metrics.png')

In [ ]:
# Save SHAP values and feature order dictionaries to file
with open("shap_dict.pickle", "wb") as file:
    pickle.dump(shap_results, file, pickle.HIGHEST_PROTOCOL)

with open("feat_order.pickle", "wb") as file:
    pickle.dump(feat_order, file, pickle.HIGHEST_PROTOCOL)

# Also save test folds
with open("fold_index_dict.pickle", "wb") as file:
    pickle.dump(fold_index, file, pickle.HIGHEST_PROTOCOL)

In [7]:
# Load in dictionaries
with open("feat_order.pickle", "rb") as f:
    feat_dict = pickle.load(f)
with open("shap_dict.pickle", "rb") as f:
    shap_dict = pickle.load(f)
with open("fold_index_dict.pickle", "rb") as f:
    fold_index_dict = pickle.load(f)

In [ ]:
# # Set up X and y
# X = adata.copy()
# y = adata.obs['orig_cancer_label']
# groups_col = adata.obs['PatientNumber']
# fold_indices_dict = {}

# # Generate cross-validation splits using StratifiedGroupKFold
# sgkf = StratifiedGroupKFold(n_splits=CV, shuffle = True, random_state = RANDOM_STATE)
# # Loop through each fold
# for i, (train_index, test_index) in enumerate(sgkf.split(X, y, groups_col)):
#     print(f'i: {i}')
#     X_train, X_test = X[train_index], X[test_index]
#     y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
#     # Store train and test indices in a dictionary by fold
#     fold_indices_dict[i] = {'train': train_index, 'test': test_index}

# # Also save test folds
# with open("fold_indices_dict.pickle", "wb") as file:
#     pickle.dump(fold_indices_dict, file, pickle.HIGHEST_PROTOCOL)

# fold_indices_dict

In [8]:
# Loop through to plot
num_features = [20, 50, 100, 200, 300, 5000]
feat_sel_list = ['dge', 'seurat_v3', 'pearson_residuals', 'seurat', 'cell_ranger', 'random_all_genes'] #, 'random_per_num'
for curr_method in feat_sel_list:
    print(f'curr_method: {curr_method}')
    for curr_num in num_features:
        print(f'curr_num: {curr_num}')
        shap_vals_df = plot_feat_importance(adata, curr_method, curr_num,
                                       #   feat_order, shap_results, fold_index)
                                            feat_dict, shap_dict, fold_index_dict,
                                           'outputs/20241118/')


curr_method: dge
curr_num: 20
fold: 0
fold: 1
fold: 2
fold: 3
fold: 4
curr_num: 50
fold: 0
fold: 1
fold: 2
fold: 3
fold: 4
curr_num: 100
fold: 0
fold: 1
fold: 2
fold: 3
fold: 4
curr_num: 200
fold: 0
fold: 1
fold: 2
fold: 3
fold: 4
curr_num: 300
fold: 0
fold: 1
fold: 2
fold: 3
fold: 4
curr_num: 5000
fold: 0
fold: 1
fold: 2
fold: 3
fold: 4
curr_method: seurat_v3
curr_num: 20
fold: 0
fold: 1
fold: 2
fold: 3
fold: 4
curr_num: 50
fold: 0
fold: 1
fold: 2
fold: 3
fold: 4
curr_num: 100
fold: 0
fold: 1
fold: 2
fold: 3
fold: 4
curr_num: 200
fold: 0
fold: 1
fold: 2
fold: 3
fold: 4
curr_num: 300
fold: 0
fold: 1
fold: 2
fold: 3
fold: 4
curr_num: 5000
fold: 0
fold: 1
fold: 2
fold: 3
fold: 4
curr_method: pearson_residuals
curr_num: 20
fold: 0
fold: 1
fold: 2
fold: 3
fold: 4
curr_num: 50
fold: 0
fold: 1
fold: 2
fold: 3
fold: 4
curr_num: 100
fold: 0
fold: 1
fold: 2
fold: 3
fold: 4
curr_num: 200
fold: 0
fold: 1
fold: 2
fold: 3
fold: 4
curr_num: 300
fold: 0
fold: 1
fold: 2
fold: 3
fold: 4
curr_num: 5000


In [9]:
# calculate Jaccard coefficient
df = calc_jaccard_coeff(feat_sel_list, num_features, feat_dict, 5)
df.tail()
#df.to_csv('jaccard_df.csv')

fold: 0
fold: 1
fold: 2
fold: 3
fold: 4


,method1,method2,num_features,fold,jaccard_coeff
445,random_all_genes,dge,5000,4,0.111246
446,random_all_genes,seurat_v3,5000,4,0.118568
447,random_all_genes,pearson_residuals,5000,4,0.118944
448,random_all_genes,seurat,5000,4,0.116695
449,random_all_genes,cell_ranger,5000,4,0.116445


In [10]:
#df.sort_values('jaccard_coeff', ascending = False).head(20)

# Average Jaccard coefficients across folds
#df = pd.read_csv('jaccard_df.csv', index_col = 0)
display(df.head())
df_grp = df.groupby(['method1', 'method2', 'num_features'])['jaccard_coeff'].agg(['mean', 'std'])
df_grp.to_csv('jaccard_coeff_grp_20241118.csv')

,method1,method2,num_features,fold,jaccard_coeff
0,seurat_v3,dge,20,0,0.000000
1,pearson_residuals,dge,20,0,0.000000
2,pearson_residuals,seurat_v3,20,0,0.111111
3,seurat,dge,20,0,0.000000
4,seurat,seurat_v3,20,0,0.000000


In [12]:
# Calculate jaccard between folds of same method/num features
feat_sel_list = ['dge', 'seurat_v3', 'pearson_residuals', 'seurat', 'cell_ranger']
df2 = calc_jaccard_coeff_btw_folds(feat_sel_list, num_features, feat_dict, 5)
df2.to_csv('jaccard_df_btw_folds_20241118.csv')

curr_method: dge


NameError: name 'fold' is not defined

In [ ]:
df2.sort_values('jaccard_coeff', ascending = False).head(20)

In [ ]:
# Apply pipeline to run RandomizedSearchCV with highly variable features